In [31]:
import keras
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib
import collections
import multiprocessing
import yaml
import gensim

from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from gensim.corpora.dictionary import Dictionary
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout,Activation
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Masking
from keras.models import model_from_yaml
from gensim.models import Word2Vec

In [32]:
# loading data
def loadsst(path):
    xs = []
    ys = []
    file1 = open(path, 'r')
    Lines = file1.readlines()
    for line in Lines:
        soup = line.split()
        ys.append(int(soup[0].lstrip('(')))
        tokens = []
        for chunk in soup[2:]:
            if not chunk.endswith(")"):
                continue
            tokens.append(chunk.rstrip(')'))
        xs.append(tokens)
    return xs, ys

In [33]:
ssttrainxs, ssttrainys = loadsst("../comp0090/trainDevTestTrees_PTB/trees/train.txt")
sstvalidxs, sstvalidys = loadsst("../comp0090/trainDevTestTrees_PTB/trees/dev.txt")
ssttestxs, ssttestys = loadsst("../comp0090/trainDevTestTrees_PTB/trees/test.txt")
print("data successfully loaded")

data successfully loaded


In [34]:
# Create a dictionary
# Combine all data first because we need to use one dictionary
total_x = np.concatenate((ssttrainxs,sstvalidxs,ssttestxs))

VOCAB_SIZE = 5000
counter = collections.Counter()
maxlen = 0
#count the maximum length of sentences in the set
for i in range(total_x.shape[0]):
    words = total_x[i]
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

#create the word2index model
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

#apply the word2index model to the set and get the fixed-length onehot format set
xs, ys = [], []
for i in range(total_x.shape[0]):
    words = total_x[i]
    wids = [word2index[word] for word in words]
    xs.append(wids)
X = pad_sequences(xs, maxlen=56)

In [39]:
WORD2VEC_MODEL = '../comp0090/GoogleNews-vectors-negative300.bin.gz'
EMBED_SIZE = 300
word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary = True)

# embedding function
def embedding(token):
# If a token is not in our vocabulary, we use UNK to represent it.
    if token in word2vec.vocab:
        token = token
    else:
        token = "UNK"
    return word2vec[token]

# Define embedding weights
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    embedding_weights[index, :] = embedding(word)

In [40]:
#split the train set, validation set and test set
train_siz = 8544
val_siz = 1101
test_siz = 2201
train_x = X[:train_siz,:]
valid_x = X[train_siz:train_siz+val_siz,:]
test_x = X[train_siz+val_siz:,:]

# process labels dataset
train_y = keras.utils.to_categorical(ssttrainys,num_classes=5)
valid_y = keras.utils.to_categorical(sstvalidys,num_classes=5)
test_y = keras.utils.to_categorical(ssttestys,num_classes=5)

In [44]:
# DEFINE all parameters here for tuning
pool_flag = 'max'
embedding_fix = False
layer_sizes = [128, 80, 20]

vocab_dim = 300
input_length = 56
n_epoch = 30
batch_size = 128
lr = 0.0001

In [45]:
# Training process
cpu_count = multiprocessing.cpu_count() # 4

# define the early_stopping function
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10)
#define the model
model = Sequential()
# add the embedding
model.add(Embedding(output_dim = vocab_dim,
                    input_dim = VOCAB_SIZE + 1,
                    mask_zero = True,
                    weights = [embedding_weights],
                    input_length = input_length,
                    trainable = not(embedding_fix)))
#add pooling layer
model.add(Masking(mask_value = 0.0))
if pool_flag == 'mean':
    model.add(GlobalAveragePooling1D())
else:
    model.add(GlobalMaxPooling1D())

In [50]:
#add layers of MLP
for i in layer_sizes:
    model.add(Dense(units = i, activation = 'relu'))
model.add(Dense(units = 5, activation = 'softmax'))

#using cross entropy loss and Adam optimizer
adam = keras.optimizers.Adam(lr = lr, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
#use fit function to do gradient descent, save the accuracy and loss of each epoch in history
history = model.fit(train_x, train_y, batch_size = batch_size, epochs = n_epoch, verbose = 1, validation_data = (valid_x, valid_y))

print ("Train set")
score = model.evaluate(train_x, train_y,batch_size = batch_size)
print ("Validation set")
score = model.evaluate(valid_x, valid_y,batch_size = batch_size)
print ("Test set")
score = model.evaluate(test_x, test_y,batch_size = batch_size)

#use yaml to save the model and weights on the drive for prediction later
yaml_string = model.to_yaml()
with open('../comp0090/mlp.yml', 'w') as outfile:
    outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
model.save_weights('../comp0090/mlp.h5')

Epoch 1/30
67/67 [==============================] - 2s 28ms/step - loss: 1.5741 - accuracy: 0.2688 - val_loss: 1.5679 - val_accuracy: 0.2525
Epoch 2/30
67/67 [==============================] - 2s 25ms/step - loss: 1.5566 - accuracy: 0.2882 - val_loss: 1.5563 - val_accuracy: 0.2897
Epoch 3/30
67/67 [==============================] - 2s 25ms/step - loss: 1.5437 - accuracy: 0.3166 - val_loss: 1.5436 - val_accuracy: 0.3297
Epoch 4/30
67/67 [==============================] - 2s 26ms/step - loss: 1.5298 - accuracy: 0.3392 - val_loss: 1.5315 - val_accuracy: 0.3361
Epoch 5/30
67/67 [==============================] - 2s 33ms/step - loss: 1.5113 - accuracy: 0.3580 - val_loss: 1.5122 - val_accuracy: 0.3488
Epoch 6/30
67/67 [==============================] - 2s 28ms/step - loss: 1.4853 - accuracy: 0.3666 - val_loss: 1.4838 - val_accuracy: 0.3515
Epoch 7/30
67/67 [==============================] - 2s 25ms/step - loss: 1.4468 - accuracy: 0.3791 - val_loss: 1.4594 - val_accuracy: 0.3524
Epoch 8/30
67

In [54]:
# DEFINE all parameters here for tuning
pool_flag = 'mean'
embedding_fix = False
rate = 2
layer_sizes = [128*rate, 80*rate, 20*rate]

vocab_dim = 300
input_length = 56
n_epoch = 500
batch_size = 128
lr = 0.0001

# Training process
cpu_count = multiprocessing.cpu_count() # 4

#define the early_stopping function
#early_stopping = EarlyStopping(monitor=’val_loss’,patience=10)
#define the model
model = Sequential()

#add the embedding
model.add(Embedding(output_dim=vocab_dim,
                    input_dim = VOCAB_SIZE + 1,
                    mask_zero = True,
                    weights = [embedding_weights],
                    input_length = input_length,
                    trainable = not(embedding_fix)))

#add pooling layer
model.add(Masking(mask_value = 0.0))
if pool_flag == 'mean':
    model.add(GlobalAveragePooling1D())
else:
    model.add(GlobalMaxPooling1D())

#add layers of MLP
for i in layer_sizes:
    model.add(Dense(units = i, activation = 'relu'))
model.add(Dense(units = 5, activation = 'softmax'))

#using cross entropy loss and Adam optimizer
rms = keras.optimizers.RMSprop(lr = lr, rho = 0.9, epsilon = None, decay = 0.0)
model.compile(loss = 'categorical_crossentropy', optimizer = rms,metrics = ['accuracy'])
#use fit function to do gradient descent, save the accuracy and loss of each epoch in history
history = model.fit(train_x, train_y, batch_size = batch_size, epochs = n_epoch, verbose = 1, validation_data = (valid_x, valid_y))

Epoch 1/500
67/67 [==============================] - 1s 19ms/step - loss: 1.5827 - accuracy: 0.2676 - val_loss: 1.5750 - val_accuracy: 0.2534
Epoch 2/500
67/67 [==============================] - 1s 17ms/step - loss: 1.5691 - accuracy: 0.2776 - val_loss: 1.5719 - val_accuracy: 0.2534
Epoch 3/500
67/67 [==============================] - 1s 18ms/step - loss: 1.5662 - accuracy: 0.2800 - val_loss: 1.5691 - val_accuracy: 0.2534
Epoch 4/500
67/67 [==============================] - 1s 21ms/step - loss: 1.5642 - accuracy: 0.2925 - val_loss: 1.5661 - val_accuracy: 0.2543
Epoch 5/500
67/67 [==============================] - 2s 24ms/step - loss: 1.5598 - accuracy: 0.2980 - val_loss: 1.5611 - val_accuracy: 0.3061
Epoch 6/500
67/67 [==============================] - 1s 20ms/step - loss: 1.5560 - accuracy: 0.3103 - val_loss: 1.5570 - val_accuracy: 0.3143
Epoch 7/500
67/67 [==============================] - 1s 17ms/step - loss: 1.5501 - accuracy: 0.3242 - val_loss: 1.5513 - val_accuracy: 0.2788
Epoch 

67/67 [==============================] - 1s 18ms/step - loss: 0.9300 - accuracy: 0.6208 - val_loss: 1.6579 - val_accuracy: 0.3960
Epoch 116/500
67/67 [==============================] - 1s 18ms/step - loss: 0.9183 - accuracy: 0.6239 - val_loss: 1.6606 - val_accuracy: 0.3833
Epoch 117/500
67/67 [==============================] - 1s 17ms/step - loss: 0.9186 - accuracy: 0.6217 - val_loss: 1.6854 - val_accuracy: 0.3887
Epoch 118/500
67/67 [==============================] - 1s 22ms/step - loss: 0.9152 - accuracy: 0.6257 - val_loss: 1.6573 - val_accuracy: 0.3933
Epoch 119/500
67/67 [==============================] - 1s 21ms/step - loss: 0.9138 - accuracy: 0.6286 - val_loss: 1.6424 - val_accuracy: 0.3924
Epoch 120/500
67/67 [==============================] - 1s 19ms/step - loss: 0.9091 - accuracy: 0.6344 - val_loss: 1.7168 - val_accuracy: 0.3878
Epoch 121/500
67/67 [==============================] - 1s 20ms/step - loss: 0.9156 - accuracy: 0.6271 - val_loss: 1.6583 - val_accuracy: 0.3906
Epoch 

Epoch 172/500
67/67 [==============================] - 1s 17ms/step - loss: 0.7727 - accuracy: 0.7015 - val_loss: 1.9564 - val_accuracy: 0.3896
Epoch 173/500
67/67 [==============================] - 1s 18ms/step - loss: 0.7771 - accuracy: 0.7019 - val_loss: 1.9594 - val_accuracy: 0.3869
Epoch 174/500
67/67 [==============================] - 1s 18ms/step - loss: 0.7713 - accuracy: 0.7040 - val_loss: 1.9584 - val_accuracy: 0.3960
Epoch 175/500
67/67 [==============================] - 1s 18ms/step - loss: 0.7651 - accuracy: 0.7056 - val_loss: 1.9294 - val_accuracy: 0.3851
Epoch 176/500
67/67 [==============================] - 1s 17ms/step - loss: 0.7638 - accuracy: 0.7070 - val_loss: 1.9388 - val_accuracy: 0.3778
Epoch 177/500
67/67 [==============================] - 1s 18ms/step - loss: 0.7641 - accuracy: 0.6987 - val_loss: 1.9539 - val_accuracy: 0.3851
Epoch 178/500
67/67 [==============================] - 1s 19ms/step - loss: 0.7647 - accuracy: 0.7100 - val_loss: 1.9792 - val_accuracy:

Epoch 285/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5396 - accuracy: 0.8133 - val_loss: 2.6975 - val_accuracy: 0.3678
Epoch 286/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5448 - accuracy: 0.8074 - val_loss: 2.6466 - val_accuracy: 0.3715
Epoch 287/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5339 - accuracy: 0.8130 - val_loss: 2.8259 - val_accuracy: 0.3597
Epoch 288/500
67/67 [==============================] - 1s 18ms/step - loss: 0.5367 - accuracy: 0.8118 - val_loss: 2.7021 - val_accuracy: 0.3860
Epoch 289/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5353 - accuracy: 0.8119 - val_loss: 2.8551 - val_accuracy: 0.3406
Epoch 290/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5267 - accuracy: 0.8119 - val_loss: 2.7068 - val_accuracy: 0.3778
Epoch 291/500
67/67 [==============================] - 1s 17ms/step - loss: 0.5247 - accuracy: 0.8200 - val_loss: 2.7035 - val_accuracy:

Epoch 342/500
67/67 [==============================] - 1s 18ms/step - loss: 0.4204 - accuracy: 0.8563 - val_loss: 3.0876 - val_accuracy: 0.3506
Epoch 343/500
67/67 [==============================] - 1s 22ms/step - loss: 0.4247 - accuracy: 0.8578 - val_loss: 3.0764 - val_accuracy: 0.3651
Epoch 344/500
67/67 [==============================] - 1s 19ms/step - loss: 0.4155 - accuracy: 0.8581 - val_loss: 3.1099 - val_accuracy: 0.3524
Epoch 345/500
67/67 [==============================] - 1s 17ms/step - loss: 0.4138 - accuracy: 0.8620 - val_loss: 3.1008 - val_accuracy: 0.3533
Epoch 346/500
67/67 [==============================] - 2s 23ms/step - loss: 0.4135 - accuracy: 0.8606 - val_loss: 3.1889 - val_accuracy: 0.3370
Epoch 347/500
67/67 [==============================] - 2s 23ms/step - loss: 0.4097 - accuracy: 0.8610 - val_loss: 3.1723 - val_accuracy: 0.3379
Epoch 348/500
67/67 [==============================] - 1s 21ms/step - loss: 0.4147 - accuracy: 0.8574 - val_loss: 3.0920 - val_accuracy:

Epoch 399/500
67/67 [==============================] - 1s 17ms/step - loss: 0.3113 - accuracy: 0.8944 - val_loss: 3.5743 - val_accuracy: 0.3515
Epoch 400/500
67/67 [==============================] - 1s 18ms/step - loss: 0.3120 - accuracy: 0.8930 - val_loss: 3.6303 - val_accuracy: 0.3433
Epoch 401/500
67/67 [==============================] - 2s 27ms/step - loss: 0.3009 - accuracy: 0.9012 - val_loss: 3.6012 - val_accuracy: 0.3415
Epoch 402/500
67/67 [==============================] - 2s 24ms/step - loss: 0.3039 - accuracy: 0.8997 - val_loss: 3.6263 - val_accuracy: 0.3515
Epoch 403/500
67/67 [==============================] - 2s 27ms/step - loss: 0.3028 - accuracy: 0.9000 - val_loss: 3.6475 - val_accuracy: 0.3379
Epoch 404/500
67/67 [==============================] - 2s 24ms/step - loss: 0.2997 - accuracy: 0.9026 - val_loss: 3.5877 - val_accuracy: 0.3588
Epoch 405/500
67/67 [==============================] - 1s 16ms/step - loss: 0.2961 - accuracy: 0.9029 - val_loss: 3.7791 - val_accuracy:

Epoch 456/500
67/67 [==============================] - 1s 18ms/step - loss: 0.2189 - accuracy: 0.9263 - val_loss: 4.1986 - val_accuracy: 0.3397
Epoch 457/500
67/67 [==============================] - 1s 18ms/step - loss: 0.2224 - accuracy: 0.9267 - val_loss: 4.1830 - val_accuracy: 0.3424
Epoch 458/500
67/67 [==============================] - 1s 17ms/step - loss: 0.2183 - accuracy: 0.9278 - val_loss: 4.2235 - val_accuracy: 0.3597
Epoch 459/500
67/67 [==============================] - 1s 18ms/step - loss: 0.2187 - accuracy: 0.9251 - val_loss: 4.2495 - val_accuracy: 0.3479
Epoch 460/500
67/67 [==============================] - 1s 19ms/step - loss: 0.2125 - accuracy: 0.9297 - val_loss: 4.2676 - val_accuracy: 0.3497
Epoch 461/500
67/67 [==============================] - 1s 19ms/step - loss: 0.2123 - accuracy: 0.9288 - val_loss: 4.2987 - val_accuracy: 0.3479
Epoch 462/500
67/67 [==============================] - 1s 18ms/step - loss: 0.2139 - accuracy: 0.9274 - val_loss: 4.3684 - val_accuracy:

In [55]:
print ("Train set")
score = model.evaluate(train_x, train_y,batch_size = batch_size)
print ("Validation set")
score = model.evaluate(valid_x, valid_y,batch_size = batch_size)
print ("Test set")
score = model.evaluate(test_x, test_y,batch_size = batch_size)

Train set
67/67 [==============================] - 0s 5ms/step - loss: 0.1905 - accuracy: 0.9346
Validation set
9/9 [==============================] - 0s 5ms/step - loss: 4.8490 - accuracy: 0.3206
Test set
18/18 [==============================] - 0s 5ms/step - loss: 4.4255 - accuracy: 0.3552


In [87]:
def word2array(sentance):
    xs, ys = [], []
    for i in range(1):
        words = re.compile("[a-z|\’]+", re.I).findall(sentance)
        wids = [word2index[word] for word in words]
        xs.append(wids)
    X = pad_sequences(xs, maxlen = 56)
    return X

#forward propogation to make the prediction
def MLP_predict(string):
    print ('loading model......') #load the model
    with open('../comp0090/mlp.yml', 'r') as f:
        yaml_string = yaml.load(f)
    model = model_from_yaml(yaml_string)

    print ('loading weights......') #Load the weights
    model.load_weights('../comp0090/mlp.h5')
    rms = keras.optimizers.RMSprop(lr = 0.0001, rho = 0.9, epsilon = None, decay = 0.0)
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = rms, 
                  metrics = ['accuracy'])
    
    data = word2array(string)
    data.reshape(1, -1)
    
    #print data
    result = model.predict_classes(data)
    print (result) # [[1]]
    if result[0] == 0:
        print ('Very Negative')
    elif result[0] == 1:
        print ('Slightly Negative')
    elif result[0] == 2:
        print ('Neutral')
    elif result[0] == 3:
        print ('Slightly Positive')
    else:
        print ('Very Positive')

In [ ]:
MLP_predict('The lighthearted tone and smart pacing allow for unwavering entertainment, plenty of natural humor, and Spielbergs knack for sympathizing with antiheroes.')

#Score:4/5 https://www.rottentomatoes.com/m/catch_me_if_you_can/reviews

In [ ]:
MLP_predict('The movie is a good time. It not only produces happiness, it is happy.')

#Score:5/5 https://www.rottentomatoes.com/m/catch_me_if_you_can/reviews

In [ ]:
MLP_predict('Offers mild fun but never as much as its animated 60s-retro opening credits portend.')

#Score:3/5 https://www.rottentomatoes.com/m/catch_me_if_you_can/reviews

In [ ]:
MLP_predict('Its insulting to its source material, its insulting to fans of that source material, and its insulting to its audiences intelligence.')

#Score:0.75/5 https://www.rottentomatoes.com/m/pixels/reviews

In [ ]:
MLP_predict('Bad movie Pixels is just the latest pock mark on Adam Sandlers still downtrending career, only noteworthy in its unprecedented slothfulness.')

#Score:1/5 https://www.rottentomatoes.com/m/pixels/reviews